### Tabu search

Z uporabo metahevrističnega algoritma preverjamo našo hipotezo o grafih z največjo sigma nepravilnostjo.

Opis algoritma:

- Glej pdf iz spletne - stran 7
- Soseščina bo predstavljena kot grafi, kjer vozlišču odvzamemo povezavo in do dve povezavi dodamo. 
- Tabu list shranjuje vozlisca, ki smo jih spreminjali
- tu je zaustavitveni pogoj v primeru, ko je prva rešitev najboljša, kar število iteracij

In [1]:
# importing necessary libraries
import random
import math
from itertools import combinations
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# function for generating random neighbour of graph G
def random_neighbour(G):
    # get node
    N = G.vertices()[randrange(G.order())]
    # remove one edge of node N
    if len(G.neighbors(N)) != 0:
        G.delete_edge(N, G.neighbors(N)[randrange(len(list(G.neighbors(N))))])
    
    # add up to two edges
    k = 0 # max number of iterations counter
    # i is 0, 1 or 2, depending on probability (0: .4, 1: .45, 2: .15)
    i = 0
    if random.random() > .4:
        i += 1
        if random.random() > .75:
            i += 1
    j = 0 # counter for number of added edges
    while k < 8 and j < i:
        k += 1
        # new node
        V = G.vertices()[randrange(G.order())]
        if V == N or V in G.neighbors(N): # if node is N or is already connected to N, we skip it
            continue
        G.add_edge(N, V)
        
        # is triangle free
        if G.triangles_count() != 0:
            G.delete_edge(N, V) # if its not triangle free, we remove the edge that we added
        else: 
            j += 1
            continue

    return G # we return random neighbour

In [2]:
# sigma total irr, our f(s)
def sigma_irr(G):
    return sum((u - v) ** 2 for u, v in combinations(G.degree(), 2))

In [3]:
# tabu search - main function (our initial solution, number of tweaks, length of tabu list, max_iterations_without_improvement)
def tabu_search(initial_G, m, l, max_iterations_without_improvement):
    G = initial_G.copy() # set G 
    best = G.copy() # set current best
    tabu_list = [] # initialize tabu list
    num_of_it_without_improvement = 0 # counter for iterations without improvement

    while num_of_it_without_improvement < max_iterations_without_improvement:
        # update tabu list
        if len(tabu_list) > l:
            tabu_list.pop(0)
        
        # new solution m times
        G_mod = random_neighbour(G) # first solution
        replace = False # check if its allowed (if it was recently used)
        if any(G_mod_mod.is_isomorphic(g) for g in tabu_list):
            replace = True

        for i in range(m-1):
            G_mod_mod = random_neighbour(G) # additional option

            # check, whether option is allowed and if its better or previous one isn't allowed
            if not(any(G_mod_mod.is_isomorphic(g) for g in tabu_list)) and (sigma_irr(G_mod) < sigma_irr(G_mod_mod) or replace == True):
                G_mod = G_mod_mod.copy()
                replace = False
        
        # update G 
        G = G_mod.copy()

        # add to tabu list
        tabu_list.append(G)

        # check if its better than current best
        if sigma_irr(G) > sigma_irr(best):
            best = G.copy()
            num_of_it_without_improvement = 0 # reset counter
        else: # count this iteration as "failed"
            num_of_it_without_improvement += 1

    return best, G # return best solution and last option

### Pridobivanje prve rešitve

In [4]:
# generate star graph with max sigma total irr. (in center there can be multiple vertices)
def generate_max_sigma_star_graph(n):
    
    # generate star graph with or order n and c central vertices
    def generate_star_graph(n, c):
        graph = {} # dict. of neighbours
        i = 0
        for k in range(n): 
            i += 1
            if i <= c: # if vertex is central, set outer vertices as neighbors
                graph[k] = list(range(c, n))
            else: # else vertex is outer, set central vertices as neighbors
                graph[k] = list(range(c))
        
        return Graph(graph)


    max_sigma = 0 # for keeping maximum sigma value
    max_sigma_graphs = [] # list of graphs with maximum sigma value

    # check all graphs, where number of central nodes varies
    for c in range(1, round(n/2)):
        G = generate_star_graph(n, c) # get star graph
        sigma = sigma_irr(G) # get sigma value of star graph

        if sigma > max_sigma: # update if its the best
            max_sigma_graphs = [G]
            max_sigma = sigma
        elif sigma == max_sigma: # if equal, just append
            max_sigma_graphs.append(G)
    
    return max_sigma_graphs # return all graphs with max sigma value

### Poganjanje algoritmov - zaenkrat samo testiranje

In [7]:
def raziskovanje_okolice(n, m, l, max_it, s = 13):

    # check if there is better solution for any orded
    found_better_anywhere = False

    for j in range(s, n+1):
        found = False # check for better solution in this order
        graphs_found = [] # keep count of better graphs

        # make initial graph
        initial_G = generate_max_sigma_star_graph(j)[0]
        max_sigma = sigma_irr(initial_G)

        # run metaheuristic method 100 times
        for i in range(100):
            Best, G = tabu_search(initial_G, m, l, max_it)
            if sigma_irr(Best) > max_sigma:
                found = True
                graphs_found.append(Best)

        # print if better solution is found for thist order
        if found == True:
            found_better_anywhere = True
            print(f"{j} - FOUND BETTER!")
        else:
            print(f"{j} -False")
    
    # print for all orders
    if found_better_anywhere == True:
        print("FOUND BETTER OPTION SOMEWHERE")
    else:
        print("THEORY HOLDS")

In [8]:
n = 500 # check graphs up to order n
l = round(n) # size of tabu list
m = 4 # number of searches inside one loop
max_it = 3 # number of iterations without improvement
raziskovanje_okolice(n, m, l, max_it)

13 -False
14 -False
15 -False
16 -False
17 -False
18 -False
19 -False
20 -False
21 -False
22 -False
23 -False
24 -False
25 -False
26 -False
27 -False
28 -False
29 -False
30 -False
31 -False
32 -False
33 -False
34 -False
35 -False
36 -False
37 -False
38 -False
39 -False
40 -False
41 -False
42 -False
43 -False
44 -False
45 -False
46 -False
47 -False
48 -False
49 -False
50 -False
51 -False
52 -False
53 -False
54 -False
55 -False
56 -False
57 -False
58 -False
59 -False
60 -False
61 -False
62 -False
63 -False
64 -False
65 -False
66 -False
67 -False
68 -False
69 -False
70 -False
71 -False
72 -False
73 -False
74 -False
75 -False
76 -False
77 -False
78 -False
79 -False
80 -False
81 -False
82 -False
83 -False
84 -False
85 -False
86 -False
87 -False
88 -False
89 -False
90 -False
91 -False
92 -False
93 -False
94 -False
95 -False
96 -False
97 -False
98 -False
99 -False
100 -False
101 -False
102 -False
103 -False
104 -False
105 -False
106 -False
107 -False
108 -False
109 -False
110 -False
111 -Fals

In [ ]:
n = 1100 # check graphs up to order n
s = 1000 # check graphs from order s
raziskovanje_okolice(n, m, l, max_it, s)

n = 2050 # check graphs up to order n
s = 2000 # check graphs from order s
raziskovanje_okolice(n, m, l, max_it, s)

n = 3025 # check graphs up to order n
s = 3000 # check graphs from order s
raziskovanje_okolice(n, m, l, max_it, s)

n = 4010 # check graphs up to order n
s = 4000 # check graphs from order s
raziskovanje_okolice(n, m, l, max_it, s)

n = 5005 # check graphs up to order n
s = 5000 # check graphs from order s
raziskovanje_okolice(n, m, l, max_it, s)